In [1]:
import numpy as np, matplotlib.pyplot as plt
import sys; sys.path.append("../src2")
from pendulum import *
import pickle

In [ ]:
# load optimal control data
trajs = np.load("energy_walks_alpha.npy")
fig1, ax1 = plt.subplots(1)
plot_traj(trajs[-1][:,1:5], ax=ax1, arm=True)
ax1.set_xlabel(r"$x$")
ax1.set_ylabel(r"$y$")
fig2, ax2 = plt.subplots(1)
plot_controls(trajs[-1][:,0], trajs[-1][:,-1], ax=ax2)
"""
for traj in trajs:
    t = traj[:,0]
    x = traj[:,1:5]
    u = traj[:,-1]
    plot_traj(x, ax=ax1, alpha=0.1)
    plot_controls(t, u, ax=ax2, alpha=0.1)
    
"""
ax1.set_ylim(-1,1)    
ax2.set_ylim(-1,1)
ax2.set_xlabel(r"$\tau$")
ax2.set_ylabel(r"$u$")

In [ ]:
fig1.savefig("traj_db.pdf", bbox_inches='tight')
fig2.savefig("cont_db.pdf", bbox_inches='tight')

In [ ]:
# format data for neural network
db = np.vstack(trajs)
nt = db.shape[0]
db = db[:,[1,2,3,4,-1]]
db = data(db, [0,1,2,3], [4])
nt

In [ ]:
# instantiate neural network
#nn = mlp([4, 20, 1])

In [ ]:
# train
n = nt
nn.train(db.i[:n,:], db.o[:n,:], epo=10000, lr=1e-3, gpu=True)

In [ ]:
# plot loss
fig, ax = plt.subplots(1)
ax.plot(nn.ltst, "k--")
ax.plot(nn.ltrn, "k-")
ax.set_yscale('log')

#ax.set_xscale('log')

In [ ]:
# instantiate controller
cont = mlp_controller(nn)

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(traj[:,0], traj[:,-1], "k--")
ax.plot(traj[:,0], cont.predict(traj[:,1:5]), "k-")
ax.set_aspect("equal")

In [ ]:
# simulate with neural network control
x0 = [0,0,np.pi,0]
xf = np.array([0,0,0,0])
dyn  = dynamics(x0,xf, 0)
t, x, u = dyn.propagate_controlled(14, cont)

In [ ]:
n = np.argwhere(np.linalg.norm(x - xf, axis=1) < 0.178)[-1][0]
t = t[:n]
x = x[:n,:]
u = u[:n]
ax = plot_traj(x, arm=True, n=500)
ax = plot_states(t, x, u)

In [ ]:
np.trapz(abs(u), t)

In [ ]:
#pickle.dump(nn, open("nn.p", "wb"))

In [ ]:
nn = pickle.load(open("nn.p", "rb"))

In [ ]:
%matplotlib

In [ ]:
nn.train = mlp.train

In [ ]:
nn.train

In [ ]:
res

In [ ]:
np.vstack((trajs)).shape

In [ ]:
x0[2:]

In [ ]:
nom = trajs[-1]

In [ ]:
np.trapz(abs(nom[:,-1]), nom[:,0])

In [ ]:
nom[:,0]

In [ ]:
t